In [1]:
import os 
import torch 
import torch.nn as nn 
import pickle 
import numpy as np

from os.path import abspath, join, dirname
sys.path.append('../')

from models import (Encoder, Probe, AC, Probe, LatentForward)
from utils import unpickle 

### Models e.t.c

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ac = AC(din=256, nk=45, nact=2).to(device)
enc = Encoder(100 * 100, 256).to(device)
forward = LatentForward(256, 2).to(device)
a_probe = Probe(256, 2).to(device)
model = torch.load('../model.p', map_location=torch.device('cpu'))

NameError: name 'mixer' is not defined

In [8]:

# a_probe.load_state_dict(model['a_probe'])
enc.load_state_dict(model['enc'])
enc.eval()
a_probe.eval()

RuntimeError: Error(s) in loading state_dict for Encoder:
	Missing key(s) in state_dict: "m.1.weight", "m.1.bias", "m.1.running_mean", "m.1.running_var", "m.3.weight", "m.3.bias", "m.4.weight", "m.4.bias", "m.4.running_mean", "m.4.running_var", "m.6.weight", "m.6.bias". 
	Unexpected key(s) in state_dict: "mixer.PerPatchMLP.mlp.weight", "mixer.PerPatchMLP.mlp.bias", "mixer.MixerStack.0.0.layer_norm.weight", "mixer.MixerStack.0.0.layer_norm.bias", "mixer.MixerStack.0.0.mlp1.weight", "mixer.MixerStack.0.0.mlp1.bias", "mixer.MixerStack.0.0.mlp2.weight", "mixer.MixerStack.0.0.mlp2.bias", "mixer.MixerStack.0.1.layer_norm.weight", "mixer.MixerStack.0.1.layer_norm.bias", "mixer.MixerStack.0.1.mlp3.weight", "mixer.MixerStack.0.1.mlp3.bias", "mixer.MixerStack.0.1.mlp4.weight", "mixer.MixerStack.0.1.mlp4.bias", "mixer.MixerStack.1.0.layer_norm.weight", "mixer.MixerStack.1.0.layer_norm.bias", "mixer.MixerStack.1.0.mlp1.weight", "mixer.MixerStack.1.0.mlp1.bias", "mixer.MixerStack.1.0.mlp2.weight", 

## Data set and preliminaries


In [6]:
klusters_predictor = unpickle("../kmeans.p")
dataset = unpickle('../dataset.p')

X, A, ast, est = dataset['X'], dataset['A'], dataset['ast'], dataset['est']
latent_states = []
predicted_grounded_states = []
for i in range(0, 100000, 256):
    with torch.no_grad():
        _latent_state = enc(torch.FloatTensor(X[i:i + 256]).to(device))
        latent_states += _latent_state.cpu().numpy().tolist()
        predicted_grounded_states += a_probe(_latent_state).cpu().numpy().tolist()

predicted_grounded_states = np.array(predicted_grounded_states)
grounded_states = np.array(ast[:len(latent_states)])
latent_states = np.array(latent_states)

TypeError: forward() missing 1 required positional argument: 'gt'